This notebook plots the MEarth light curves for GJ 1132b, downloaded from the MEarth data release page.

In [ ]:
import glob, numpy as np, matplotlib.pyplot as plt
from astropy.io.ascii import read
from lightkurve import LightCurve
import astropy.units as u
from astropy.time import Time
from exoatlas.visualizations.ink_errorbar import ink_errorbar, one2another

MEarth-South's `tel13` and `tel16` were the two telescopes that continuously monitored the brightnes sof GJ1132 over many years, including both in-transit and out-of-transit observations.

In [ ]:
telescopes = [13, 16]
files = [f'mearth-light-curves/2MASSJ10145184-4709244_tel{t}_2014-2019.txt' for t in telescopes]
files

We want to trim out the high-cadence transit observations, so they don't too strongly affect our binned averages with lots and lots of observations all at one time (to say nothing of the transit itself).

In [ ]:
def remove_high_cadence(lc):
    '''
    Trim out high-cadence (= targeted transit) observations.
    '''
    
    # identify which moments include >N data points in <threshold days
    N  = 30
    threshold = 0.5/24
    time_for_N = lc.time.value[N:] - lc.time.value[:-N] 

    # idenitify which points are follow
    followed_by_high_cadence = np.nonzero(time_for_N < threshold)[0]
    
    # create an array of which datapoints to include
    ok = np.ones_like(lc.time.value).astype(np.bool)

    # remove high cadence points
    ok[followed_by_high_cadence] = False
    ok[followed_by_high_cadence + N] = False
    return lc[ok]

There are both the `uncorrected` raw light curves, and the `corrected` light curves where we've made an effort to correct for various instrumental systematics like meridian flips or the precipitable water-vapor common mode. Let's look at both of them.

In [ ]:
def plot_epochs():
    visits = np.arange(1,6)
    transit_epochs = np.array([862.19257, 1020.19808, 1077.21151, 1080.46896, 1083.72617])# + 2457000
    colors = ['cornflowerblue', 
              'crimson', 
              'seagreen', 
              'chocolate',
              'orchid']
    for v, e, c in zip(visits, transit_epochs, colors):
        plt.axvline(e, color=c, zorder=-10, label=f'Visit {v}')
        #plt.text(e, 1.02, f'Visit {v}')

In [ ]:
colnames = dict(Uncorrected='Mag', Corrected='Corr_Mag') 

for option in colnames:

    lcs= {}
    for t, f in zip(telescopes, files):
        lc = read(f, data_start=54, names='BJD           Mag         e_Mag       tExp    DMag    FWHM   Ellip Airmass X         Y         Angle   Sky      Peak  S  V  R F CM        Corr_Mag'.split())
        lcs[t] = LightCurve(time=Time(lc['BJD']-2457000.0 , format='btjd'), 
                            flux=10**(-0.4*lc[colnames[option]]), 
                            flux_err=1.086*lc['e_Mag'])

    
    fi, ax = plt.subplots(1,1, figsize=(8.5,2.5))
    colors = {13:'#1f77b4', 16: '#ff7f0e'}# {13:'brown', 16: 'dimgray'}
    for t in lcs:
        lc = remove_high_cadence(lcs[t])
        binned = lc.bin(1*u.day, aggregate_func=np.nanmedian).remove_nans()
        ink_errorbar(x=binned.time.value, 
                     y=binned.flux.value,
                     xerr=np.zeros_like(binned.time.value),
                     yerr=binned.flux_err.value, 
                     c=1/binned.flux_err.value**2,
                     cmap=one2another('white', colors[t]),
                     vmax=1e6, vmin=1e3)
    span = 0.025
    plt.ylim(1-span, 1+span)
    #plt.legend()

    plt.xlabel('Time - 2457000 [BTJD days]')
    plt.ylabel(f'Relative Flux')
    plot_epochs()
    plt.legend(bbox_to_anchor=(1,1), frameon=False)

    plt.tight_layout()
    plt.savefig(f'GJ1132-error-bars-{option}.pdf')

Cool! The 120-130 day rotation period seems to show up particularly clearly in the last few cycles from ~1000-1750 days. Overall, the peak-to-peak modulation amplitude is about 1-2%.